In [1]:
## Import libraries 
import pandas as pd

import glob

from pathlib import Path
import csv

import os
import requests
import json

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
### Code for Looping through all SEC files in Folder 

# new_df = pd.DataFrame()
# for filename in all_files:
#     if filename == 0:
#         new_df = read_ftd_file(csv_path=filename ,cusip_number=cusip_number,symbol=symbol)
#     else:
#         new_df2 = read_ftd_file(csv_path=filename ,cusip_number=cusip_number,symbol=symbol)
#         new_df = concat_df(new_df,new_df2)
        
# new_df

In [3]:
## Set Test Path 
csv_path = '../SEC_Files_CSV/cnsfails202108a.csv'
# raw_data = pd.read_csv(
#     Path(csv_path),
#     #skiprows = 1,
#     #header=None,
#     index_col=None
# )

# # Drop last two rows 
# # by selecting all rows except last two rows
# df = raw_data.iloc[:-2 , :]

# Header = "Header"
# new_data = df.rename(columns={header:Header})
# new_data = new_data.Header.str.split("|",expand=True)

# new_data = new_data.rename(columns={0:'Date',1:'CUSIP',2:'SYMBOL',3:'QUANTITY_FAILS',4:'DESCRIPTION',5:'PRICE'})
# new_data.to_csv('../FilesTemp/test_ftd_clean.csv')
# new_data 


# raw_data_delimit = pd.read_csv(
#     Path(csv_path),
#     delimiter='|',  #some rows don't work 
#     index_col=None
# )

# # data_dropped = raw_data.dropna()
# # data_dropped #45821
# raw_data_delimit.iloc[35:36]  # 47728 rows 

In [4]:
## Declare Header Variables to make Code work 
header = "SETTLEMENT DATE|CUSIP|SYMBOL|QUANTITY (FAILS)|DESCRIPTION|PRICE"

## Make FTD read Function 
def read_ftd_file(csv_file_path):
    df = pd.read_csv(
    Path(csv_file_path),
    index_col=None
    )
    
    df = df.iloc[:-2 , :]

    Header = "Header"
    df = df.rename(columns={header:Header})
    df = df.Header.str.split("|",expand=True)

    df = df.rename(columns={0:'Date',1:'CUSIP',2:'SYMBOL',3:'QUANTITY_FAILS',4:'DESCRIPTION',5:'PRICE'})
    
    ## Description column has commas in data and is not relevant anyways 
    ## Price column has errors and missing data. Source daily price from IEX later
    df.drop(['DESCRIPTION','PRICE'], axis=1,inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    return df 

In [5]:
# test_function = read_ftd_file(csv_path)
# test_function

In [6]:
## Set Path variables for SEC data files 
path = r'C:\Users\watso\OneDrive\Projects\FTD_Project_Personal\SEC_Files_CSV' # USE YOUR PATH 
all_files = glob.glob(path + "/*.csv")

ftd_df = pd.DataFrame()

for filename in all_files:
    if filename == 0:
        ftd_df = read_ftd_file(filename)
    else:
        ftd_df2 = read_ftd_file(filename)
        ftd_df = pd.concat([ftd_df,ftd_df2],axis='rows')

ftd_df

,CUSIP,SYMBOL,QUANTITY_FAILS
Date,,,
2021-01-04,B38564108,EURN,19305
2021-01-04,D18190898,DB,11259
2021-01-04,G00748106,STWO,17643
2021-01-04,G00748122,STWOU,3141
2021-01-04,G0083D104,ACEVW,42372
...,...,...,...
2021-09-30,98985W102,ZYME,67530
2021-09-30,98985X100,ZY,92721
2021-09-30,98986M103,ZYXI,325


In [7]:
ftd_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1090911 entries, 2021-01-04 to 2021-09-30
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   CUSIP           1090911 non-null  object
 1   SYMBOL          1090911 non-null  object
 2   QUANTITY_FAILS  1090911 non-null  object
dtypes: object(3)
memory usage: 33.3+ MB


In [12]:
cusip_symbol_df = ftd_df[['SYMBOL','CUSIP']]
cusip_symbol_df = cusip_symbol_df.reset_index(drop=True)
cusip_symbol_df = cusip_symbol_df.drop_duplicates()
cusip_symbol_df = cusip_symbol_df.sort_values('SYMBOL')
cusip_symbol_df = cusip_symbol_df.reset_index(drop=True)
#cusip_symbol_df = cusip_symbol_df.set_index('SYMBOL')
cusip_symbol_df

,SYMBOL,CUSIP
0,,Q0819E102
1,5116REORGPYMNT,674215116
2,8105REORGPYMNT,M82618105
3,A,00846U101
4,AA,013872106
...,...,...
18194,ZYXI,98986M103
18195,ZZHGY,98955F105
18196,ZZLL,98880P202
18197,ZZZOD,98959W203


In [11]:
## Export DFs 
ftd_df.to_csv('../Resources/ftd_all_data.csv')
cusip_symbol_df.to_csv('../Resources/symbol_all_list.csv')

In [9]:
## Error Finding - Load individual files to find errors as necessary 

csv_path = '../SEC_Files_CSV/cnsfails202104b.csv'
test_df = read_ftd_file(csv_path)
test_df

,CUSIP,SYMBOL,QUANTITY_FAILS
Date,,,
2021-04-15,D1668R123,DDAIF,363
2021-04-15,D18190898,DB,19435
2021-04-15,G0084W101,ADNT,90
2021-04-15,G01022105,CXRXF,35
2021-04-15,G0103T105,ACAC,12347
...,...,...,...
2021-04-30,98981L100,ZMDTF,729
2021-04-30,989825104,ZURVY,4352
2021-04-30,98983V106,ZUO,301


In [10]:
test_df[6].isnull().values.sum()

KeyError: 6

In [ ]:
test_df2 = test_df.reset_index()

In [ ]:
## Code to locate indexes of entries giving errors 
## DMYQWS entries give errors due to a '|' in DESCRIPTION creating an extra column

nullcolumn = test_df2[6]
length_ = len(nullcolumn)

for i in range(length_):
    var = nullcolumn[i]
    if var == None:
        continue
    else:
        print(i)

In [ ]:
test_df.iloc[58602]